In [1]:
import os
import os.path as op
# Directorio con los archivos de entrada
input_dir = 'C:\\Users\\aag23\\Documents\\9ec61f3d95be7870bb9f9cda2e68d28e'
# Hacemos un listado de los archivos en el directorio
import os
input_files = os.listdir(input_dir)
# vemos que son ficheros con extensión .grib y los pasamos a formato .nc con cdo
input_files = [f for f in input_files if f.endswith('.grib')]
# Creamos un directorio de salida
output_dir = 'C:\\Users\\aag23\\Documents\\9ec61f3d95be7870bb9f9cda2e68d28e\\output'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
# Procesamos los ficheros
'''for f in input_files:
    input_file = op.join(input_dir, f)
    output_file = op.join(output_dir, f.replace('.grib', '.nc'))
    cmd = 'cdo -f nc copy ' + input_file + ' ' + output_file
    os.system(cmd)
    '''

"for f in input_files:\n    input_file = op.join(input_dir, f)\n    output_file = op.join(output_dir, f.replace('.grib', '.nc'))\n    cmd = 'cdo -f nc copy ' + input_file + ' ' + output_file\n    os.system(cmd)\n    "

In [2]:
# leemos un fichero .nc y motramos su contenido
import xarray as xr
input_file = op.join(output_dir, '1month_mean_Global_ea_tp_197908_v02.nc')
ds = xr.open_dataset(input_file)
print(ds)
# Mostramos las variables
print(ds.data_vars)
# Mostramos las dimensiones
print(ds.dims)
# Mostramos los atributos
print(ds.attrs)
# Mostramos las dimensiones de la variable
print(ds['tp'].dims)
# Mostramos los atributos de la variable
print(ds['tp'].attrs)
# Mostramos los valores de la variable
print(ds['tp'].values)
# cerramos el fichero
ds.close()


<xarray.Dataset> Size: 4MB
Dimensions:  (time: 1, lon: 1440, lat: 721)
Coordinates:
  * time     (time) datetime64[ns] 8B 1979-08-01T06:00:00
  * lon      (lon) float64 12kB 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * lat      (lat) float64 6kB 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
Data variables:
    tp       (time, lat, lon) float32 4MB ...
Attributes:
    CDI:          Climate Data Interface version 2.0.4 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    institution:  European Centre for Medium-Range Weather Forecasts
    history:      Thu Jan 02 12:30:15 2025: cdo -f nc copy /mnt/c/Users/aag23...
    CDO:          Climate Data Operators version 2.0.4 (https://mpimet.mpg.de...
Data variables:
    tp       (time, lat, lon) float32 4MB ...
FrozenMappingWarningOnValuesAccess({'time': 1, 'lon': 1440, 'lat': 721})
{'CDI': 'Climate Data Interface version 2.0.4 (https://mpimet.mpg.de/cdi)', 'Conventions': 'CF-1.6', 'institution': 'European Centre for Medium-Range Weather 

In [3]:
# Postprocesado de ficheros .nc a uno único
# que solo contengan datos para españa
# contenga todos los instantes temporales
# latitud: 25, 45.0
lat_min = 25
lat_max = 45
# longitud: -30.0, 5.0
lon_min = -30
lon_max = 5
# Directorio de salida
output_dir_post = 'C:\\Users\\aag23\\Documents\\9ec61f3d95be7870bb9f9cda2e68d28e\\output_post'
if not os.path.exists(output_dir_post):
    os.makedirs(output_dir_post)
# Procesamos los ficheros con xarray
input_files = os.listdir(output_dir)
for f in input_files:
    input_file = op.join(output_dir, f)
    ds = xr.open_dataset(input_file, engine='netcdf4')
    # Seleccionamos los datos para españa
    # por un lado de 330 a 360 y por otro de 0 a 5
    ds = ds.sel(lon=slice(360+lon_min, 360)).combine_first(ds.sel(lon=slice(0, lon_max)))
    # Seleccionamos los datos para la latitud
    ds = ds.sel(lat=slice(lat_max, lat_min))
    # Pasamos la longitud de 0-360 a -180, 180
    ds = ds.assign_coords(lon=(ds.lon + 180) % 360 - 180)
    # reordenamos las longitudes
    ds = ds.sortby(ds.lon)
    # Añadimos atributos a lon
    # serian   :standard_name = "longitude";
    # :long_name = "longitude";
    # :units = "degrees";
    # :axis = "X";
    ds['lon'].attrs['standard_name'] = 'longitude'
    ds['lon'].attrs['long_name'] = 'longitude'
    ds['lon'].attrs['units'] = 'degrees'
    ds['lon'].attrs['axis'] = 'X'
    # Guardamos el fichero
    output_file = op.join(output_dir_post, f)
    ds.to_netcdf(output_file)
    ds.close()


In [4]:
# Leemos los ficheros de salida
output_files = os.listdir(output_dir_post)
output_files = [f for f in output_files if f.endswith('.nc')]
# Creamos un fichero único
output_file = op.join(output_dir_post, 'spain.nc')
ds = xr.open_mfdataset([op.join(output_dir_post, f) for f in output_files])
ds.to_netcdf(output_file)
ds.close()